In [ ]:
import tensorflow as tf
import keras
import sklearn
import scikeras
import sys
import numpy as np

print("GPU Available: ", tf.test.is_gpu_available())
tf.__version__, keras.__version__, sklearn.__version__, scikeras.__version__, np.__version__, sys.version

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from keras import layers, saving
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasClassifier

data_path = "/kaggle/input/filtered-clouds/CCSN_filtered_224/"
test_path = data_path+"test"
name_suffix = ""
SEED = 43

In [ ]:
keras.utils.set_random_seed(SEED)

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)

train_dataset = keras.utils.image_dataset_from_directory(
    data_path+"train",
    label_mode="categorical",
    batch_size=None,
    image_size=IMAGE_SIZE,
)


In [ ]:
test_dataset = keras.utils.image_dataset_from_directory(
    test_path,
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
)

In [ ]:
labels_text = sorted(os.listdir(test_path))
labels_dict = {i: name for i, name in zip(range(len(labels_text)), labels_text)}
labels_text_to_num = {name: i for i, name in zip(range(len(labels_text)), labels_text)}
labels_dict

In [ ]:
@saving.register_keras_serializable("Custom")
class PreprocessingLayerResnet(layers.Layer):
    def __init__(self, **kwargs):
        super(PreprocessingLayerResnet, self).__init__(**kwargs)

    def get_config(self):
        config = super(PreprocessingLayerResnet, self).get_config()
        return config

    def call(self, inputs, training=None):
        outputs = keras.applications.resnet.preprocess_input(inputs)
        outputs.set_shape(inputs.shape)
        return outputs

@saving.register_keras_serializable("Custom")
class PreprocessingLayerXception(layers.Layer):
    def __init__(self, **kwargs):
        super(PreprocessingLayerXception, self).__init__(**kwargs)

    def get_config(self):
        config = super(PreprocessingLayerXception, self).get_config()
        return config

    def call(self, inputs, training=None):
        outputs = keras.applications.xception.preprocess_input(inputs)
        outputs.set_shape(inputs.shape)
        return outputs


min_augmentation_layer = keras.models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom((-0.25, 0.15)),
], name="data_augmentation")


In [ ]:
def get_y(ds):
    y = []
    for _, labels in ds:
        for label in labels:
            y.append(np.argmax(label, axis=-1))
    return y

y_train = get_y(train_dataset)
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
print(class_weights_dict)

In [ ]:
IMG_SHAPE = IMAGE_SIZE + (3,)

def create_model(dropout=0.2, optimizer_type=keras.optimizers.Adam, initial_lr=0.00005, decay_steps=5000, decay_rate=0.2,
                 hidden_neurons=128, hidden_activation='gelu', freeze_point=1/3,
                 kernel_regularizer=keras.regularizers.L2, kernel_reg_val=0.006):
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=initial_lr,
        decay_steps=decay_steps,
        decay_rate=decay_rate)
    optimizer = optimizer_type(lr_schedule)
    
    input_tensor = keras.Input(shape=IMG_SHAPE)
    x = input_tensor
    x = min_augmentation_layer(x)

    conv_base = keras.applications.convnext.ConvNeXtTiny(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
    conv_base.trainable = True
    for layer in conv_base.layers[:int(len(conv_base.layers) * freeze_point)]:
        layer.trainable =  False

    x = conv_base(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout)(x)
    if hidden_neurons > 0:
        x = layers.Dense(hidden_neurons, activation=hidden_activation, kernel_regularizer=kernel_regularizer(kernel_reg_val))(x)
        x = layers.Dropout(dropout)(x)
    predictions = layers.Dense(11, activation='softmax')(x)
    model = keras.models.Model(inputs=input_tensor, outputs=predictions)
    model.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=optimizer, metrics=['accuracy'])
    return model


In [ ]:
x_train, y_train = [], []
for img, label in train_dataset:
    x_train.append(img)
    y_train.append(label)

x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
param_grid = dict(
    dropout=np.linspace(0, 0.8, 9),
    optimizer_type=[keras.optimizers.Adam, keras.optimizers.Nadam, keras.optimizers.Adadelta],
    initial_lr=[0.001, 0.0001, 0.00005, 0.00001],
    decay_steps=[1000, 5000, 10000],
    decay_rate=[1, 0.5, 0.2, 0.1],
    hidden_neurons=[128, 256, 512],
    hidden_activation=['gelu', 'relu', 'elu', 'leaky_relu'],
    freeze_point=[1/3, 1/2, 1/4],
    kernel_regularizer=[keras.regularizers.L1, keras.regularizers.L2, keras.regularizers.L1L2],
    kernel_reg_val=np.linspace(0, 0.008, 9)
)
sklearn_model = KerasClassifier(model=create_model, epochs=30, batch_size=BATCH_SIZE, verbose=0, loss=keras.losses.CategoricalCrossentropy(), **param_grid)
rand_search = RandomizedSearchCV(
    estimator=sklearn_model, cv=3, param_distributions=param_grid, verbose=4, n_iter=10, n_jobs=1, random_state=SEED)

grid_result = rand_search.fit(x_train, y_train,
                             class_weight=class_weights_dict,
                             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
                            )

best_model = grid_result.best_estimator_

In [ ]:
cv_results_df = pd.DataFrame(grid_result.cv_results_)
cv_results_df.to_csv('rand_search.csv')

In [ ]:
print(grid_result.best_params_)
cv_results_df

In [ ]:
def show_imgs_list(imgs, labels, wrong_labels):
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 8)
    axes = *ax1, *ax2, *ax3, *ax4
    fig.set_size_inches(20, 10)
    for ax, img, label, wrong_label in zip(axes, imgs, labels, wrong_labels):
        ax.set_title(labels_dict.get(int(np.argmax(label, axis=-1)), "Error") + " vs pred: " + labels_dict.get(int(np.argmax(wrong_label, axis=-1)), "Error"))
        ax.axis("off")
        img = np.array(img)
        ax.imshow(img.astype(np.uint8))

test_imgs = []
test_labels = []
for imgs, labels in test_dataset:
    for img, label in zip(imgs, labels):
        test_imgs.append(img)
        test_labels.append(label)

x_test, y_test = np.array(test_imgs), np.array(test_labels)

In [ ]:
predicted_labels = best_model.predict(x_test)
misclassified_indices = np.argmax(predicted_labels, axis=-1) != np.argmax(y_test, axis=-1)
show_imgs_list(x_test[misclassified_indices], y_test[misclassified_indices], predicted_labels[misclassified_indices])

In [ ]:
def show_cm(model, x, y, out_name):
    y_test_pred = model.predict(x)
    y_test_pred_cls = [np.argmax(sample) for sample in y_test_pred]
    cm = confusion_matrix(y, y_test_pred_cls, labels=list(labels_dict.keys()))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(labels_dict.values()))
    disp.plot()
    plt.savefig("/kaggle/working/test_confusion_matrix_"+out_name+"_"+name_suffix+".png")
    plt.show()

show_cm(best_model, x_test, [np.argmax(label) for label in y_test], "convnext")